This code read all train dicom files and transform every feature into column in Dataframe **except images** and then merge it with train csv.

It can be reproduced with test data.

Also it separate arrays features like 'ImageType' or 'PixelSpacing' into new columns.

Then it save result as pkl file that i'm using it for the rest of my work.

It can be usfull for your work then i share it.


# Train Part

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
from pydicom import dcmread
from pydicom.data import get_testdata_files
import glob, os
from collections import defaultdict
import tqdm
import gc
import seaborn as sns
import ast

In [2]:
col_name = ['ImageType', 'SOPInstanceUID', 'Modality', 'Manufacturer','ManufacturerModelName', 'PatientName', 'PatientID', 'PatientSex', 'DeidentificationMethod',
'BodyPartExamined', 'SliceThickness', 'KVP', 'SpacingBetweenSlices', 'GantryDetectorTilt', 'TableHeight', 'RotationDirection', 'XRayTubeCurrent', 'ConvolutionKernel',                 
'PatientPosition', 'StudyInstanceUID', 'SeriesInstanceUID', 'StudyID', 'InstanceNumber', 'ImagePositionPatient', 'ImageOrientationPatient', 'FrameOfReferenceUID',          
'PositionReferenceIndicator', 'SliceLocation', 'SamplesPerPixel', 'PhotometricInterpretation', 'Rows', 'Columns', 'PixelSpacing','BitsAllocated','BitsStored',                    
'HighBit', 'PixelRepresentation','WindowCenter','WindowWidth','RescaleIntercept','RescaleSlope']

df = pd.DataFrame(columns=col_name)
my_dict = defaultdict(list)

for name in tqdm.tqdm(glob.glob('/kaggle/input/osic-pulmonary-fibrosis-progression/train/*/*')):
    ds = pydicom.read_file(name)
    for i in col_name :
        if i in ds :
            my_dict[i].append(str(ds[i].value))
        else:
            my_dict[i].append(np.nan)
    df = pd.concat([df, pd.DataFrame(my_dict)], ignore_index = True)
    del my_dict
    my_dict = defaultdict(list)
gc.collect()

for name in tqdm.tqdm(glob.glob('/kaggle/input/osic-pulmonary-fibrosis-progression/test/*/*')):
    ds = pydicom.read_file(name)
    for i in col_name :
        if i in ds :
            my_dict[i].append(str(ds[i].value))
        else:
            my_dict[i].append(np.nan)
    df = pd.concat([df, pd.DataFrame(my_dict)], ignore_index = True)
    del my_dict
    my_dict = defaultdict(list)
gc.collect()
#     ds = pydicom.read_file(name)
#     df = pd.DataFrame(ds.values())
#     df[0] = df[0].apply(lambda x: pydicom.dataelem.DataElement_from_raw(x) if isinstance(x, pydicom.dataelem.RawDataElement) else x)
#     df['name'] = df[0].apply(lambda x: x.name)
#     df['value'] = df[0].apply(lambda x: x.value)
#     df = df[['name', 'value']]

#     df = df.set_index('name').T.reset_index(drop=True)
#     df.drop('Pixel Data', axis = 1, inplace = True)

100%|██████████| 1261/1261 [01:51<00:00, 11.28it/s]


0

In [3]:
train = pd.concat([pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv'), pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')], ignore_index = True)

In [4]:
# train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
Data = df.merge(train , left_on='PatientID', right_on='Patient')

type_dict_all = ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI', 'HELIX', 'CT_SOM5 SEQ', 'SECONDARY', 'DERIVED', 'JP2K LOSSY 6:1', 'VOLUME', 'OTHER', 'CSA MPR', 'CSAPARALLEL', 
                'CSA RESAMPLED', 'REFORMATTED', 'AVERAGE', 'CT_SOM7 SPI DUAL', 'STD', 'SNRG', 'DET_AB']
for i in type_dict_all:
    Data[i] = np.nan
for index, row in tqdm.tqdm(Data.iterrows()):
    for i in type_dict_all:
        if i in Data.loc[index, 'ImageType'] :
            Data.loc[index, i] = 1
            


303286it [32:39, 154.77it/s]


In [5]:
Data.fillna(0, inplace = True)

Data.loc[Data['ImagePositionPatient'] == 0, 'ImagePositionPatient'] = '[0,0,0]'
Data['ImagePositionPatient'] = Data['ImagePositionPatient'].apply(ast.literal_eval)
Data[['ImagePositionPatient_x','ImagePositionPatient_y', 'ImagePositionPatient_z']] = pd.DataFrame(Data.ImagePositionPatient.tolist(), index= Data.index)

Data.loc[Data['ImageOrientationPatient'] == 0, 'ImageOrientationPatient'] = '[0,0,0,0,0,0]'
Data['ImageOrientationPatient'] = Data['ImageOrientationPatient'].apply(ast.literal_eval)
Data[['ImageOrientationPatient_a','ImageOrientationPatient_b', 'ImageOrientationPatient_c', 'ImageOrientationPatient_d', 'ImageOrientationPatient_e', 'ImageOrientationPatient_f']] = pd.DataFrame(Data.ImageOrientationPatient.tolist(), index= Data.index)

tmp1 = Data[['ImagePositionPatient_x','ImagePositionPatient_y', 'ImagePositionPatient_z', 'ImageOrientationPatient_a','ImageOrientationPatient_b', 'ImageOrientationPatient_c']]
tmp1.columns = ['x','y','z','a','b','c']

tmp1['Cos'] = 'red'
tmp2 = Data[['ImagePositionPatient_x','ImagePositionPatient_y', 'ImagePositionPatient_z', 'ImageOrientationPatient_d','ImageOrientationPatient_e', 'ImageOrientationPatient_f']]
tmp2.columns = ['x','y','z','a','b','c']
tmp2['Cos'] = 'blue'

cos = pd.concat([tmp1, tmp2], ignore_index = True)
cos['width'] = 10

cos[['a','b','c']] = cos[['a','b','c']] * 200

Data.loc[Data['PixelSpacing'] == 0, 'PixelSpacing'] = '[0,0]'
Data['PixelSpacing'] = Data['PixelSpacing'].apply(ast.literal_eval)
Data[['PixelSpacing_row','PixelSpacing_column']] = pd.DataFrame(Data.PixelSpacing.tolist(), index= Data.index)

Data.to_pickle('output_data.pkl')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
Data.head(10)

,ImageType,SOPInstanceUID,Modality,Manufacturer,ManufacturerModelName,PatientName,PatientID,PatientSex,DeidentificationMethod,BodyPartExamined,...,ImagePositionPatient_y,ImagePositionPatient_z,ImageOrientationPatient_a,ImageOrientationPatient_b,ImageOrientationPatient_c,ImageOrientationPatient_d,ImageOrientationPatient_e,ImageOrientationPatient_f,PixelSpacing_row,PixelSpacing_column
0,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.54870549743787326149757258476993295366,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-155.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281
1,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.54870549743787326149757258476993295366,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-155.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281
2,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.54870549743787326149757258476993295366,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-155.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281
3,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.54870549743787326149757258476993295366,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-155.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281
4,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.54870549743787326149757258476993295366,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-155.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281
5,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.54870549743787326149757258476993295366,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-155.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281
6,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.54870549743787326149757258476993295366,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-155.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281
7,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.54870549743787326149757258476993295366,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-155.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281
8,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.54870549743787326149757258476993295366,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-155.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281
9,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.124514977742315273882283417037267436849,CT,Philips,Brilliance 16,ID00123637202217151272140,ID00123637202217151272140,,Table;,Chest,...,-20.526262,-139.5,1.0,0.0,0.0,0.0,1.0,0.0,0.613281,0.613281


# Test Part

In [7]:
col_name = ['ImageType', 'SOPInstanceUID', 'Modality', 'Manufacturer','ManufacturerModelName', 'PatientName', 'PatientID', 'PatientSex', 'DeidentificationMethod',
'BodyPartExamined', 'SliceThickness', 'KVP', 'SpacingBetweenSlices', 'GantryDetectorTilt', 'TableHeight', 'RotationDirection', 'XRayTubeCurrent', 'ConvolutionKernel',                 
'PatientPosition', 'StudyInstanceUID', 'SeriesInstanceUID', 'StudyID', 'InstanceNumber', 'ImagePositionPatient', 'ImageOrientationPatient', 'FrameOfReferenceUID',          
'PositionReferenceIndicator', 'SliceLocation', 'SamplesPerPixel', 'PhotometricInterpretation', 'Rows', 'Columns', 'PixelSpacing','BitsAllocated','BitsStored',                    
'HighBit', 'PixelRepresentation','WindowCenter','WindowWidth','RescaleIntercept','RescaleSlope']

df = pd.DataFrame(columns=col_name)
my_dict = defaultdict(list)

for name in tqdm.tqdm(glob.glob('/kaggle/input/osic-pulmonary-fibrosis-progression/test/*/*.dcm')):
    ds = pydicom.read_file(name)
    for i in col_name :
        if i in ds :
            my_dict[i].append(str(ds[i].value))
        else:
            my_dict[i].append(np.nan)
    df = pd.concat([df, pd.DataFrame(my_dict)], ignore_index = True)
    del my_dict
    my_dict = defaultdict(list)
gc.collect()

100%|██████████| 1261/1261 [00:13<00:00, 90.76it/s]


560

In [8]:
test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
sub = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')
sub['Patient'] = sub['Patient_Week'].apply(lambda x:x.split('_')[0])
sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub =  sub[['Patient','Weeks','Patient_Week']]
test = sub.merge(test.drop('Weeks', axis=1), on="Patient")

Data = df.merge(test , left_on='PatientID', right_on='Patient')

type_dict_all = ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI', 'HELIX', 'CT_SOM5 SEQ', 'SECONDARY', 'DERIVED', 'JP2K LOSSY 6:1', 'VOLUME', 'OTHER', 'CSA MPR', 'CSAPARALLEL', 
                'CSA RESAMPLED', 'REFORMATTED', 'AVERAGE', 'CT_SOM7 SPI DUAL', 'STD', 'SNRG', 'DET_AB']
for i in type_dict_all:
    Data[i] = np.nan
for index, row in tqdm.tqdm(Data.iterrows()):
    for i in type_dict_all:
        if i in Data.loc[index, 'ImageType'] :
            Data.loc[index, i] = 1

184106it [13:26, 228.37it/s] 


In [9]:
Data.fillna(0, inplace = True)

Data.loc[Data['ImagePositionPatient'] == 0, 'ImagePositionPatient'] = '[0,0,0]'
Data['ImagePositionPatient'] = Data['ImagePositionPatient'].apply(ast.literal_eval)
Data[['ImagePositionPatient_x','ImagePositionPatient_y', 'ImagePositionPatient_z']] = pd.DataFrame(Data.ImagePositionPatient.tolist(), index= Data.index)

Data.loc[Data['ImageOrientationPatient'] == 0, 'ImageOrientationPatient'] = '[0,0,0,0,0,0]'
Data['ImageOrientationPatient'] = Data['ImageOrientationPatient'].apply(ast.literal_eval)
Data[['ImageOrientationPatient_a','ImageOrientationPatient_b', 'ImageOrientationPatient_c', 'ImageOrientationPatient_d', 'ImageOrientationPatient_e', 'ImageOrientationPatient_f']] = pd.DataFrame(Data.ImageOrientationPatient.tolist(), index= Data.index)

tmp1 = Data[['ImagePositionPatient_x','ImagePositionPatient_y', 'ImagePositionPatient_z', 'ImageOrientationPatient_a','ImageOrientationPatient_b', 'ImageOrientationPatient_c']]
tmp1.columns = ['x','y','z','a','b','c']

tmp1['Cos'] = 'red'
tmp2 = Data[['ImagePositionPatient_x','ImagePositionPatient_y', 'ImagePositionPatient_z', 'ImageOrientationPatient_d','ImageOrientationPatient_e', 'ImageOrientationPatient_f']]
tmp2.columns = ['x','y','z','a','b','c']
tmp2['Cos'] = 'blue'

cos = pd.concat([tmp1, tmp2], ignore_index = True)
cos['width'] = 10

cos[['a','b','c']] = cos[['a','b','c']] * 200

Data.loc[Data['PixelSpacing'] == 0, 'PixelSpacing'] = '[0,0]'
Data['PixelSpacing'] = Data['PixelSpacing'].apply(ast.literal_eval)
Data[['PixelSpacing_row','PixelSpacing_column']] = pd.DataFrame(Data.PixelSpacing.tolist(), index= Data.index)

Data.to_pickle('output_data_test.pkl')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
Data.head()

,ImageType,SOPInstanceUID,Modality,Manufacturer,ManufacturerModelName,PatientName,PatientID,PatientSex,DeidentificationMethod,BodyPartExamined,...,ImagePositionPatient_y,ImagePositionPatient_z,ImageOrientationPatient_a,ImageOrientationPatient_b,ImageOrientationPatient_c,ImageOrientationPatient_d,ImageOrientationPatient_e,ImageOrientationPatient_f,PixelSpacing_row,PixelSpacing_column
0,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.11631487706382795974542148315653548560,CT,Philips,Brilliance 64,ID00423637202312137826377,ID00423637202312137826377,,Table;,Chest,...,-56.5,949.300049,1.0,0.0,0.0,0.0,1.0,0.0,0.693359,0.693359
1,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.11631487706382795974542148315653548560,CT,Philips,Brilliance 64,ID00423637202312137826377,ID00423637202312137826377,,Table;,Chest,...,-56.5,949.300049,1.0,0.0,0.0,0.0,1.0,0.0,0.693359,0.693359
2,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.11631487706382795974542148315653548560,CT,Philips,Brilliance 64,ID00423637202312137826377,ID00423637202312137826377,,Table;,Chest,...,-56.5,949.300049,1.0,0.0,0.0,0.0,1.0,0.0,0.693359,0.693359
3,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.11631487706382795974542148315653548560,CT,Philips,Brilliance 64,ID00423637202312137826377,ID00423637202312137826377,,Table;,Chest,...,-56.5,949.300049,1.0,0.0,0.0,0.0,1.0,0.0,0.693359,0.693359
4,"['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']",2.25.11631487706382795974542148315653548560,CT,Philips,Brilliance 64,ID00423637202312137826377,ID00423637202312137826377,,Table;,Chest,...,-56.5,949.300049,1.0,0.0,0.0,0.0,1.0,0.0,0.693359,0.693359
